In [1]:
import pandas as pd 
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from collections import defaultdict
import numpy as np

In [2]:
?ShuffleSplit

In [2]:
def oob_feature_score(X,y,n_estimator=500,test_size=0.4):
    feature_names = X.columns
    rf= RandomForestRegressor(n_estimators=n_estimator)
    scores =defaultdict(list)
    avgscores =defaultdict(list)
    rs =ShuffleSplit(n_splits=10,test_size=0.4,random_state=0)
    for train_idx, test_idx in rs.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = y.iloc[train_idx], y.iloc[test_idx]
        r = rf.fit(X_train, Y_train)
        acc = r2_score(Y_test, rf.predict(X_test))
        for i in range(X.shape[1]):
            X_t = X_test.copy()
            X_t.iloc[:,i] = np.random.permutation(X_t.iloc[:,i].values)
            shuff_acc = r2_score(Y_test, rf.predict(X_t))
            scores[i].append((acc-shuff_acc)/acc)
    for i,score in scores.items():
        avgscores[i]=round(np.mean(score),6)
        
        df=pd.DataFrame({'feature':feature_names[key],'importance':value} for key, value in avgscores.items())
        
    return df



In [3]:
def gini_feature_score(X,y,n_estimator=1000,test_size=0.4):
    feature_names = X.columns
    rf= RandomForestRegressor(n_estimators=n_estimator)
    scores =defaultdict(list)
    avgscores =defaultdict(list)
    rs =ShuffleSplit(n_splits=10,test_size=0.4,random_state=0)
    for train_idx, test_idx in rs.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = y.iloc[train_idx], y.iloc[test_idx]
        r = rf.fit(X_train, Y_train)
        #acc = r2_score(Y_test, rf.predict(X_test))
        for i in range(X.shape[1]):
            scores[i].append(r.feature_importances_[i])
#         for i in range(X.shape[1]):
#             X_t = X_test.copy()
#             X_t.iloc[:,i] = np.random.permutation(X_t.iloc[:,i].values)
#             shuff_acc = r2_score(Y_test, rf.predict(X_t))
#             scores[i].append((acc-shuff_acc)/acc)
    for i,score in scores.items():
        avgscores[i]=round(np.mean(score),6)
        
        df=pd.DataFrame({'feature':feature_names[key],'importance':value} for key, value in avgscores.items())
        
    return df

In [4]:
defaultdict(list)[0].append(1)

In [5]:
ny= pd.read_csv('zipcode_level_data/ny_model_final.csv') 
#ny.rename(columns={'0':'num_of_crime_half_year'},inplace=True)


In [6]:
X = ny.drop(columns= [
'num_of_airbnb'
#,'num_of_beds'
,'avg_price'
,'total_reviews'
,'avg_location_review_score'
,'avg_review_score'
,'zipcode']
# ,'household_income:< $25,000'
# ,'household_income:$25,000-$44,999'
# ,'household_income:$45,000-$59,999'
# ,'household_income:$60,000-$99,999'
# ,'household_income:$100,000-$149,999'
# ,'household_income:$150,000-$199,999'
# ,'household_income:$200,000+'
# ,'vancancy_reason: For Rent'
# ,'vancancy_reason: Rented & Unoccupied'
# ,'vancancy_reason: For Sale Only'
# ,'vancancy_reason: Sold & Unoccupied'
# ,'vancancy_reason: For Season Recreational Or Occasional Use'
# ,'vancancy_reason: For Migrant Workers'
# ,'vancancy_reason: Vacant For Other Reasons'
# ,'C-distance_ts']
    ).fillna(0)

## Num of Airbnb

### New York

In [8]:
num_of_airbnb_score = gini_feature_score(X,ny['num_of_airbnb'])
num_of_airbnb_score.sort_values(by='importance',ascending=False).head(42)

,feature,importance
5,num_of_tourist_attractions,0.234385
2,c_distance,0.167766
28,zillow_rent_index,0.091842
18,housing_built_after_1990,0.070549
3,num_of_restaurant,0.063232
9,youth_20_to_39,0.044410
21,num_of_studio_apt,0.039596
4,num_of_market,0.038583
26,Renter_Occupied_Households,0.034351
23,num_of_2b_apt,0.022404


In [23]:
num_of_airbnb_score.to_csv('ny_feature_score_airbnbamount.csv')

In [10]:
def table_clean(df,city):
    
    drop_columns= ['num_of_airbnb','avg_price','total_reviews','avg_location_review_score','avg_review_score','zipcode']
#         ['num_of_airbnb','num_of_beds','avg_price','total_reviews','avg_location_review_score'
#                 ,'avg_review_score','zipcode',#'household_income:< $25,000','household_income:$25,000-$44,999'
#                   # ,'household_income:$45,000-$59,999','household_income:$60,000-$99,999','household_income:$100,000-$149,999'
#                    #,'household_income:$150,000-$199,999','household_income:$200,000+'
#                     'vancancy_reason: For Rent','vancancy_reason: Rented & Unoccupied'
#                 ,'vancancy_reason: For Sale Only','vancancy_reason: Sold & Unoccupied','vancancy_reason: For Season Recreational Or Occasional Use'
#                    ,'vancancy_reason: For Migrant Workers' ,'vancancy_reason: Vacant For Other Reasons']
    
    X= df.drop(columns=drop_columns)
    X.rename(columns={'0':'num_crime_in_half'},inplace=True)
#     if city=='bos':
#         X= X.drop(columns='C-distance_bc')
#     if city=='dc':
#         X= X.drop(columns='C-distance_wh')
#     if city=='ny':
#         X= X.drop(columns='C-distance_ts')
    X = X.fillna(0)
    return X

### Boston

In [11]:
bos = pd.read_csv('zipcode_level_data/bos_model_final.csv') 
X = table_clean(bos,'bos')
bos_airbnb_score = gini_feature_score(X,bos['num_of_airbnb'])
bos_airbnb_score.sort_values(by='importance',ascending=False).head(42)

,feature,importance
17,crimes_in_half_year,0.327635
21,num_of_studio_apt,0.206571
14,household_income_less_than_25000_percent,0.064134
4,num_of_market,0.057380
1,num_of_sub,0.047656
3,num_of_restaurant,0.030819
22,num_of_1b_apt,0.030546
16,employment_rate,0.026871
9,youth_20_to_39,0.019925
13,Asian,0.017677


In [24]:
bos_airbnb_score.to_csv('bos_feature_score_airbnbamount.csv')

### DC

In [27]:
dc = pd.read_csv('zipcode_level_data/dc_model_final.csv') 
X = table_clean(dc,'dc')
dc_airbnb_score = gini_feature_score(X,dc['num_of_airbnb'])
dc_airbnb_score.sort_values(by='importance',ascending=False).head(42)

,feature,importance
4,num_of_market,0.265572
9,youth_20_to_39,0.135993
3,num_of_restaurant,0.109025
17,crimes_in_half_year,0.079766
5,num_of_tourist_attractions,0.072164
22,num_of_1b_apt,0.049860
12,American_Indian_Or_Alaskan_Native,0.035070
21,num_of_studio_apt,0.026907
2,c_distance,0.025425
1,num_of_sub,0.023717


In [28]:
dc_airbnb_score.to_csv('dc_feature_score_airbnbamount.csv')

## Avg Price

### New York

In [13]:
X = table_clean(ny,'ny')
ny_price_score = gini_feature_score(X,ny['avg_price'])
ny_price_score.sort_values(by='importance',ascending=False).head(42)

,feature,importance
15,bachelor_above_percentage,0.309522
28,zillow_rent_index,0.262270
2,c_distance,0.076296
12,American_Indian_Or_Alaskan_Native,0.063067
16,employment_rate,0.038062
14,household_income_less_than_25000_percent,0.029522
18,housing_built_after_1990,0.022485
25,vancancy_reason_For_Season_Recreational_Or_Occ...,0.021404
6,num_of_travel_agency,0.017556
3,num_of_restaurant,0.015871


In [26]:
ny_price_score.to_csv('ny_feature_score_price.csv')

### DC

In [29]:
X = table_clean(dc,'dc')
dc_price_score = gini_feature_score(X,dc['avg_price'])
dc_price_score.sort_values(by='importance',ascending=False)

,feature,importance
28,zillow_rent_index,0.098424
5,num_of_tourist_attractions,0.090819
15,bachelor_above_percentage,0.062184
25,vancancy_reason_For_Season_Recreational_Or_Occ...,0.061095
14,household_income_less_than_25000_percent,0.057371
12,American_Indian_Or_Alaskan_Native,0.050609
6,num_of_travel_agency,0.049701
11,Black_Or_African_American,0.047644
20,home_value_more_than_400000,0.046315
2,c_distance,0.043222


In [31]:
dc_price_score.to_csv('dc_feature_score_price.csv')

### Boston

In [15]:
X = table_clean(bos,'bos')
bos_price_score = gini_feature_score(X,bos['avg_price'])
bos_price_score.sort_values(by='importance',ascending=False)

,feature,importance
13,Asian,0.180608
3,num_of_restaurant,0.144866
2,c_distance,0.115425
9,youth_20_to_39,0.079414
28,zillow_rent_index,0.077477
16,employment_rate,0.062682
19,home_value_less_than_400000,0.047453
15,bachelor_above_percentage,0.040194
22,num_of_1b_apt,0.022305
5,num_of_tourist_attractions,0.019781


In [30]:
bos_price_score.to_csv('bos_feature_score_price.csv')